# 前言
---
在前面的文章[《统计风险模型初探》](https://uqer.io/community/share/59a6521a57065101082e643b)里，我简单的对统计风险模型进行了大致的探索，实测下来效果一般。本文参考东方证券《风险模型提速组合优化的另一种方案》，将统计风险模型与压缩估计法进行结合，同时将得到的协方差矩阵的预测与优矿的基本面风险模型所得到的协方差矩阵的预测进行对比测试。本文有两个目的:
* 探索因子模型在求解组合优化问题上是否具有计算优势
* 比较统计风险模型与基本面风险模型孰优孰劣


In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from datetime import datetime, timedelta
from sklearn.covariance import LedoitWolf
from CAL.PyCAL import *
from quartz_extensions.Optimizer.optimize import *
cal = Calendar('CHINA.SSE')

In [ ]:
universe = DataAPI.EquGet(equTypeCD='A', listStatusCD='L,S,DE')
universe = universe['secID'].unique().tolist()

mkt = DataAPI.MktEqudAdjGet(secID=universe, beginDate='20081201', endDate='20171229', field='secID,tradeDate,closePrice')
mkt = mkt.pivot(index='tradeDate', columns='secID', values='closePrice')
mkt = mkt.pct_change()
mkt = mkt.dropna(how='all', axis=0)
mkt.index = mkt.index.str.replace('-', '')

In [ ]:
def qrm_pc(covariacne, k):
    """
    主成分法分解协方差矩阵
    输入：
        covariacne:np.mat，N*N的协方差矩阵，其中N为股票的个数
        k:int, 保留的主成分的个数

    输出：
        result:dict，记录中间变量和目标变量
        """
    tv = np.diag(covariacne)
    values, vector = np.linalg.eig(covariacne)
    values = values.astype(float)
    vectors = vector.astype(float)
    y = {'values': values, 'vectors': vectors}
    
    k = int(min(k, covariacne.shape[1]-2))
    fac_load = (np.mat(np.diag(np.sqrt(y['values'][:k])))*y['vectors'][:, :k].T).T
    fac_cov = np.eye(k)
    x_f = fac_load * fac_load.T
    x_s = tv - np.diag(x_f)
    spec_risk = np.sqrt(x_s)
    cov_mat = np.diag(x_s) + x_f

    y_s = np.diag(1 / spec_risk ** 2)
    v = fac_load
    v1 = y_s * v
    inv_cov = y_s - v1 * np.linalg.inv(np.eye(k) + v.T * v1) * v1.T

    result = {}
    result['spec_risk'] = spec_risk
    result['fac_load'] = fac_load
    result['fac_cov'] = fac_cov
    result['cov_mat'] = pd.DataFrame(cov_mat, index=covariacne.columns, columns=covariacne.columns)
    result['inv_cov'] = inv_cov
    result['pc'] = y['vectors'][:, :k]
    return result['cov_mat']

class CalCov():
    def __init__(self):
        pass
    
    @staticmethod
    def shrink_covariance(universe, trade_date, mkt):
        pre_date = (cal.advanceDate(trade_date, '-252B')).strftime('%Y%m%d')
        universe_return = mkt.loc[pre_date:trade_date, universe]
        mkt.dropna(thresh=126, axis=1)
        universe_return.fillna(0, inplace=True)
        cov_mat = LedoitWolf().fit(universe_return.as_matrix()).covariance_ 
        return pd.DataFrame(cov_mat, index=universe_return.columns, columns=universe_return.columns)
    
    @staticmethod
    def fundamental_risk_model_covariance(univese, trade_date, risk_model_type):
        raise Exception('暂不支持请自行编写')
    
    @staticmethod
    def shrink_stat_riskmdl_covariance(universe, trade_date, mkt, k):
        cov_mat = CalCov.shrink_covariance(universe, trade_date, mkt)
        return qrm_pc(cov_mat, k)

# 一、模型原理
---
* 关于统计风险模型的原理在前面的文章已经做初步的说明。此处不再赘述，这次把对协方差矩阵进行分解的代码一并提供，也算是一个小小的福利吧。其实对于压缩估计，经常使用优矿的矿友一定不会陌生，此前占据社区头条一年多的热帖[MultiFactors Alpha Model - 基于因子IC的多因子合成](https://uqer.io/community/share/57eca10d228e5b3663fac5a0)里面就介绍了这种方法。不过该文是对IC的协方差矩阵进行压缩，意义不大(在因子数量不多的情况下，我们能够直接把这个协方差矩阵算出来并保证其性质良好)。本文将这种方法应用于股票收益率的协方差矩阵上。
* 从统计的角度来说，样本协方差的估计完全依赖于数据，而先验的协方差矩阵可以来源于主观的判断、历史经验或者模型。样本协方差矩阵是无偏的，但是有大量的估计误差。而先验的协方差矩阵由于具有严格的假设，因此其设定存在偏差，但是其待估的参数少，所以估计误差也小。事实上，当股票数量很多时，样本协方差矩阵不一定可逆，即使它可逆，协方差矩阵的逆矩阵也不是协方差矩阵逆矩阵的无偏估计，实际上在正态分布假设下有：
$$E(\hat{\Sigma^{-1}}) = \frac{T}{T-M-2}\Sigma^{-1}$$
这时候，如果T表示观察的期数，M表示股票数量，这时候，如果T和M很接近，那就会给模型带来很大的偏误。
* 协方差的压缩估计量可以表示为样本协方差矩阵和压缩目标的一个线性组合，即：
$$\hat{\Sigma_{shrink}}=\rho F+(1-\rho)\hat{\Sigma}$$
其中$F$为压缩目标，$\hat{\Sigma}$为样本协方差矩阵，$\rho$为压缩强度
* 压缩估计量的关键在于压缩强度的确定，可以通过一定的损失函数来估计压缩强度。Ledoit&Wolf(2013)使用协方差矩阵压缩估计量与真实协方差矩阵之间的距离来作为损失函数，压缩强度则可以最小化损失函数求解得出。而压缩目标的选择有以下形式进行参考：
	* Ledoit(2004) 单参数形式，可以表示为方差乘以一个单位矩阵
	* Ledoit(2003b) CAPM单因子结构化模型估计
	* Ledoit(2003a) 平均相关系数形式

	本文使用了第一种形式	 

# 二、主成份个数的选择
---
事实上，统计风险模型好比是使用了统计学上的因子分析法，甚至更具体的是因子分析当中的主成份法。在前文当中，我们也留下来这样一个思考，保留的公共因子的个数k取多少合适？这里，我们使用主成份分析中的方差解释度这个概念，计算前k个最大的特征值之和占所有特征值之和的比例。

In [ ]:
# 统计自2010年以来，k的选择。
trade_date = DataAPI.TradeCalGet(exchangeCD='XSHG', beginDate='20100101', endDate='20171231')
trade_month_date = trade_date[trade_date['isMonthEnd']==1]['calendarDate'].tolist()
trade_month_date = map(lambda x: x.replace('-', ''), trade_month_date)

In [ ]:
hs300_cover_ratio = dict([(i, []) for i in [10, 20, 40, 60, 80, 100]])
for date in trade_month_date:
    dy_universe = DynamicUniverse('HS300')
    current_universe = dy_universe.preview(date)
    result = CalCov.shrink_covariance(current_universe, date, mkt)
    values, vector = np.linalg.eig(result)
    for k in [10, 20, 40, 60, 80, 100]:
        hs300_cover_ratio[k].append(float(values[:k].sum()) / float(values.sum()))

zz500_cover_ratio = dict([(i, []) for i in [10, 20, 40, 60, 80, 100]])
for date in trade_month_date:
    dy_universe = DynamicUniverse('ZZ500')
    current_universe = dy_universe.preview(date)
    result = CalCov.shrink_covariance(current_universe, date, mkt)
    values, vector = np.linalg.eig(result)
    for k in [10, 20, 40, 60, 80, 100]:
        zz500_cover_ratio[k].append(float(values[:k].sum()) / float(values.sum()))

# 由于全A计算速度过慢，我们只统计最近一年的结果
a_cover_ratio = dict([(i, []) for i in [10, 20, 40, 60, 80, 100]])
for date in trade_month_date[-12:]:
    dy_universe = DynamicUniverse('A')
    current_universe = dy_universe.preview(date)
    result = CalCov.shrink_covariance(current_universe, date, mkt)
    values, vector = np.linalg.eig(result)
    for k in [10, 20, 40, 60, 80, 100]:
        a_cover_ratio[k].append(float(values[:k].sum()) / float(values.sum()))

In [ ]:
data = pd.DataFrame(data={'k=10': [0.5202, 0.4821, 0.3440], 'k=20': [0.5962, 0.5491, 0.4091], 
                          'k=40': [0.6960, 0.6458, 0.5010], 'k=60': [0.7641, 0.7170, 0.5709], 
                          'k=80': [0.8413, 0.7723, 0.6290], 'k=100': [0.8525, 0.8161, 0.6792]},
                    index=['沪深300成份股', '中证500成份股', '全市场'])
data

,k=10,k=100,k=20,k=40,k=60,k=80
沪深300成份股,0.5202,0.8525,0.5962,0.6960,0.7641,0.8413
中证500成份股,0.4821,0.8161,0.5491,0.6458,0.7170,0.7723
全市场,0.3440,0.6792,0.4091,0.5010,0.5709,0.6290


1. 从上表的结果来看,对于沪深300成份股、中证500成份股前20个最大特征值之和占比就超过了50%；当k取100时,沪深300成分股超过了80%，但是中证500成份股依然没有达到80%，并且对于全市场来说，k取100方差解释度不足70%
2. Zura Kakushadze & Willie Yu在《Statistical Risk Models》论文当中也提到了两种关于k的选择的方法，论文附录了R语言的代码，有兴趣的读者可以参考一下

# 三、实证分析:
---
1. cvxpy在其文档中提供了一个组合优化的例子，在其例子中，详细的说明了对于一个标准的组合优化问题，使用因子模型进行组合优化的时间复杂度和使用原始的股票之间的协方差矩阵的时间复杂度的大小。不难发现，使用因子模型进行组合优化在计算上有优势。
2. 简单起见，我们使用四个例子：沪深300指数增强（成份股内，信号随机生成），沪深300增强（全市场，信号随机生成），沪深300指数增强（成份股内，Alpha信号），沪深300增强（全市场，Alpha信号）。读者也可以参考本文构造例子进行测试。本文分别测试标准压缩估计量，风险模型，和不同K取值下谱分解近似方法的优化速度和效果。组合优化问题设置如下:
$$max: f'w-\lambda w'\Sigma w$$
$$st: w'\_{active}\Sigma w_{active} <= tracking\ error$$
$$0 <=w<=0.05$$
为了避免其它约束对风险预测的影响，我们只设置跟踪误差约束，以及个股权重的上限,$\lambda$设为0.05。我们比较组合实际的跟踪误差与预先设置的跟踪误差的差别。同时为了检验我们模型对风险的预测能力，我们使用随机的组合来进行测试。再把这些结果，输入到优化器内进行求解，得到权重之后，使用优矿最新的quartz回测框架进行测试(新版回测速度很快，大家多多使用，多提建议啊)，回测的模板将在附录当中给出。下表给出了统计结果(求解时间包括数据载入的时间，实际上，在问题维度较大的情况下，我们本地测试下来，我们的优化器的性能要显著优于cvxpy+ecos)
3. 我们做两组实验,一组随机生成Alpha信号,一组使用真正的信号.通过两种不同的场景综合地进行比较

In [ ]:
summary = pd.DataFrame(columns=[['压缩估计量','因子模型','压缩估计量谱分解近似', '压缩估计量谱分解近似','压缩估计量谱分解近似','压缩估计量谱分解近似', '压缩估计量谱分解近似', '压缩估计量谱分解近似'],
                                ['-', '-', 10, 20, 40, 60, 80, 100]],
                       index=['组合真实跟踪误差','平均单期优化时间'])
summary.index.name = '沪深300成份股内(随机信号)'
summary.iloc[1, :] = [1.660, 1.1963, 0.9697, 1.3227, 1.2577, 1.3311, 1.0857, 0.6502]
summary.iloc[0, :] = [0.0901, 0.0815, 0.0905, 0.0968, 0.0909, 0.0925, 0.0912, 0.0928]
summary

压缩估计量    因子模型 压缩估计量谱分解近似                                  \
                      -       -         10      20      40      60      80   
沪深300成份股内(随机信号)                                                              
组合真实跟踪误差         0.0901  0.0815     0.0905  0.0968  0.0909  0.0925  0.0912   
平均单期优化时间           1.66  1.1963     0.9697  1.3227  1.2577  1.3311  1.0857   

                         
                    100  
沪深300成份股内(随机信号)          
组合真实跟踪误差         0.0928  
平均单期优化时间         0.6502

In [ ]:
summary = pd.DataFrame(columns=[['压缩估计量','因子模型','压缩估计量谱分解近似', '压缩估计量谱分解近似','压缩估计量谱分解近似','压缩估计量谱分解近似', '压缩估计量谱分解近似', '压缩估计量谱分解近似'],
                                ['-', '-', 10, 20, 40, 60, 80, 100]],
                       index=['跟踪误差','单期优化用时'])
summary.index.name = '沪深300成份股全A增强（随机信号）'
summary.iloc[1, :] = [20.4969, 11.1060, 26.2500, 28.1211, 27.2414, 32.6413, 29.5395, 30.3207]
summary.iloc[0, :] = [0.0715, 0.0866, 0.0764, 0.0721, 0.0706, 0.07447, 0.0687, 0.072]
summary

压缩估计量    因子模型 压缩估计量谱分解近似                             \
                          -       -         10       20       40       60   
沪深300成份股全A增强（随机信号）                                                          
跟踪误差                 0.0715  0.0866     0.0764   0.0721   0.0706  0.07447   
单期优化用时              20.4969  11.106      26.25  28.1211  27.2414  32.6413   

                                      
                         80      100  
沪深300成份股全A增强（随机信号）                    
跟踪误差                 0.0687    0.072  
单期优化用时              29.5395  30.3207

从结果来看：

- 单期优化用时与k的取值并没有呈现出明显的单调性，而且所花费的时间甚至比简单的压缩估计要多。
- 同时我们可以看到无论是使用压缩估计量，还是使用压缩估计量谱分解近似所得到协方差矩阵来控制组合跟踪误差，效果均要略优于基本面风险模型。
- 也就是说，如果单单是从组合风险的估计上，压缩估计+谱分解理论上是可行的。

那么实际上是否真的可行？本文使用两一组真实的信号来进行分析，一组为全A范围内的沪深300指数增强策略，另一组为沪深300成分股内的指数增强测了。在使用Alpha信号进行对比测试的时候,我们去除组合优化当中个股权重的上限约束,将跟踪误差约束目标值进一步下调至年化3%,然后比较压缩估计量,因子模型,以及谱分解近似(80个主成分)三个模型的效果。

In [ ]:
summary = pd.DataFrame(columns=['压缩估计量','因子模型','压缩估计量谱分解近似(80)'],
                       index=['跟踪误差','单期优化用时'])
summary.index.name = '沪深300成份股全A增强（Alpha信号）'
summary.iloc[1, :] = [215.7605, 24.1607,60.5365]
summary.iloc[0, :] = [0.0382, 0.0361, 0.0369]
summary

,压缩估计量,因子模型,压缩估计量谱分解近似(80)
沪深300成份股全A增强（Alpha信号）,,,
跟踪误差,0.0382,0.0361,0.0369
单期优化用时,215.761,24.1607,60.5365


In [ ]:
summary = pd.DataFrame(columns=['压缩估计量','因子模型','压缩估计量谱分解近似(80)'],
                       index=['跟踪误差','单期优化用时'])
summary.index.name = '沪深300成份股（Alpha信号）'
summary.iloc[1, :] = [2.6837, 2.8715, 1.4295]
summary.iloc[0, :] = [0.0410, 0.0359, 0.0415]
summary

,压缩估计量,因子模型,压缩估计量谱分解近似(80)
沪深300成份股（Alpha信号）,,,
跟踪误差,0.041,0.0359,0.0415
单期优化用时,2.6837,2.8715,1.4295


从真实的信号测试的结果来看，在全A范围内使用压缩估计所耗费得到的协方差矩阵进行组合优化所求解的时间明显要多于基本面风险模型和，而压缩矩估计+主成份分解的方法次之，基本面风险模型是最快的而且也是效果最好的，而在沪深300成分股内，则不明显。整体上来说，使用压缩矩估计+主成分分解来估计协方差矩阵是可行的。

回测部分可以使用下面的框架，对于不同的股票池，只需要修改对应不同的域,不同的权重输入即可。

In [ ]:
weights = pd.read_csv('risk_model_test/hs300_alpha_fundamental_riskmdl_m.csv', index_col=0)
weights.index = map(lambda x: str(x), weights.index)

In [ ]:
def cal_tracking_error(perf):
    perf_m = perf
    alpha = perf['returns'] - perf['benchmark_returns']
    return np.sqrt(alpha.var()*252)

In [ ]:
cal_tracking_error(perf)

0.035900007247717594

# 结论
---
1. 风险模型主要实现三个功能:组合优化,绩效归因,以及Alpha因子中性化.后面两个功能都需要基本面风险模型的相关数据.估算协方差矩阵可以使用结构化因子模型,也能通过纯统计方法(例如:压缩估计量,统计风险模型)得到。
2. 用因子模型的好处是可以大幅缩减要估计的参数数量,降低估计误差,同时也降低了组合优化过程中协方差相关计算的计算复杂度,在股票数量较多时可以将组合优化速度提升一至两个数量级,节约策略回溯调试时间。
3. 本文我们先用线性压缩估计量方法得到协方差矩阵估计 ,再对协方差矩阵进行因子分析,得到类似于基本面风险模型的结构,然后对比这三类模型使用的效果。
4. 但是从本文测试的结果来看，基本面风险模型对风险的估计略优于统计风险模型，使用真实的信号求解组合优化问题速度也会快一些，但最终差别不大。